In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/data/Ames_Housing_Price_Data_raw.csv")

In [ ]:
type_dict={
    'MiscFeature': 'Nominal',
    'Fireplaces': 'Discrete',
    'FireplaceQu': 'Ordinal',
    'PoolQC': 'Ordinal',
    'PoolArea': 'Continuous',
    'PavedDrive': 'Nominal',
    'ExterQual': 'Ordinal',
    'OverallQual': 'Ordinal',
    'OverallCond': 'Ordinal',
    'MiscVal': 'Continuous',
    'YearBuilt': 'Discrete',
    'YearRemodAdd': 'Discrete',
    'KitchenQual': 'Ordinal',
    'Fence': 'Ordinal',
    'RoofStyle': 'Nominal',
    'RoofMatl': 'Nominal',
    'Exterior1st': 'Nominal',
    'Exterior2nd': 'Nominal',
    'ExterCond': 'Ordinal',
    'MasVnrType': 'Nominal',
    'MasVnrArea': 'Continuous'
}

In [ ]:
Cond_dict={
       'Ex':5,
       'Gd':4,
       'TA':3,
       'Fa':2,
       'Po':1,
       'NA':0,
        '0':0
}

In [ ]:
df.loc[df['ExterQual'].isna(),'ExterQual']='0'
df['ExterQual']=df['ExterQual'].map(lambda x: Cond_dict[x])

df.loc[df['ExterCond'].isna(),'ExterCond']='0'
df['ExterCond']=df['ExterCond'].map(lambda x: Cond_dict[x])

df.loc[df['KitchenQual'].isna(),'KitchenQual']='0'
df['KitchenQual']=df['KitchenQual'].map(lambda x: Cond_dict[x])

df.loc[df['FireplaceQu'].isna(),'FireplaceQu']='0'
df['FireplaceQu']=df['FireplaceQu'].map(lambda x: Cond_dict[x])